In [ ]:
using CSV
using DataFrames
using PyCall

include("../src/Kraft.jl")

pd = pyimport("pandas")
kraft = pyimport("kraft")

In [ ]:
gene_x_sample_gct_file_path = "/Users/kwatme/Downloads/CCLE_expression_full.reformatted.gct"

gene_x_sample_tsv_file_path = "$gene_x_sample_gct_file_path.gene_x_sample.tsv"

# CSV.write(
#     gene_x_sample_tsv_file_path,
#     Kraft.read_gct(
#         gene_x_sample_gct_file_path,
#         "Gene",
#     );
#     delim = '\t',
# )

gmt_file_paths = [
    "c2.all.v6.2.symbols.gmt",
]

output_directory_path = "/Users/kwatme/Downloads/gsea"

gene_set_keywords = [
    "VANTVEER_BREAST_CANCER_ESR1",
    "DOANE_BREAST_CANCER_ESR1",
    "YANG_BREAST_CANCER_ESR1",
    "CHARAFE_BREAST_CANCER_LUMINAL_VS_BASAL",
]

;

In [ ]:
gene_set_x_sample_tsv_file_path = Kraft.gsea(
    gene_x_sample_tsv_file_path,
    gmt_file_paths,
    output_directory_path;
    sample_normalization_method = "-0-",
    gene_set_keywords = gene_set_keywords,
    n_required_gene_set_element = 5,
    statistic = "ks",
)

gene_set_x_sample = CSV.read(gene_set_x_sample_tsv_file_path)

In [ ]:
protein_x_sample = Kraft.read_gct(
    "/Users/kwatme/Downloads/CCLE_RPPA_20180123.gct",
    "Protein",
)

target_row_indices = findall(protein_x_sample[!, Symbol("Protein")] .== "ER-alpha")

if length(target_row_indices) != 1
    
    error("there should be only 1 target.")
    
end

target_row_index = target_row_indices[1]

target = pd.Series(
    [protein_x_sample[target_row_index, i] for i in 2:size(
        protein_x_sample,
        2,
    )],
    index = names(protein_x_sample)[2:end],
    name = "ER Protein",
)

In [ ]:
data = pd.DataFrame(
    Dict(name => gene_set_x_sample[!, name] for name in names(gene_set_x_sample)[2:end]),
    index = gene_set_x_sample[!, 1],
)

In [ ]:
kraft.make_match_panel(
    target,
    data,
    n_sampling = 0,
    n_permutation = 0,
)